In [34]:
import re
import pandas as pd
from profanity import profanity
import csv
import numpy as np
from nltk.tokenize import RegexpTokenizer
import matplotlib.pyplot as plt
from nltk.stem import SnowballStemmer
import nltk
import spacy
from textblob import TextBlob
nlp = spacy.load('de_core_news_sm')
snowStemmer = SnowballStemmer(language='german')
RAW_DATA_PATH = 'data/raw/'

pydev debugger: Unable to find real location for: C:\Users\Bennet Göttsche\AppData\Local\Temp\ipykernel_44608\2459993205.py
pydev debugger: Unable to find real location for: <frozen importlib._bootstrap_external>
pydev debugger: Unable to find real location for: <frozen codecs>
pydev debugger: Unable to find real location for: <frozen io>


Helper Functions

In [4]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [5]:
def find_longest_word(word_list):
    longest_word =  max(word_list, key=len)
    return len(longest_word)

In [6]:
def lemmatize_string(input_text):
    doc = nlp(input_text.lower())
    result = ' '.join([x.lemma_ for x in doc])
    doc = nlp(result.title())
    result = ' '.join([x.lemma_ for x in doc]).upper()
    return result

Data Extraction

In [7]:
df_Lied = pd.read_csv(RAW_DATA_PATH + 'LIED.csv', usecols=['ID','INTERPRET', 'TITEL', 'SPRACHE_DEUTSCH', 'TEXT_TEIL1', 'TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4'])
#print(df_Lied.head())

df_Chart_Position = pd.read_csv(RAW_DATA_PATH + 'CHART_POSITION.csv', usecols=['LIED_ID', 'POSITION', 'DATUM_VON', 'DATUM_BIS'])
print(df_Chart_Position.head())

#get stopword-list
with open(RAW_DATA_PATH+'Stoppwords.csv', newline='', encoding='UTF-8') as f:
    stopwords_list = list(csv.reader(f))
stopwords_list = [word.upper() for word in flatten(stopwords_list)]

   LIED_ID  POSITION   DATUM_VON   DATUM_BIS
0    26024        26  1983-04-18  1983-04-24
1     1073        46  1984-06-04  1984-06-10
2     1131        33  1984-06-18  1984-06-24
3    15819        22  1984-08-13  1984-08-19
4     7324         6  1984-08-27  1984-09-02


Data Conversion

In [8]:
df_Lied['Text'] = df_Lied['TEXT_TEIL1'].fillna('') + df_Lied['TEXT_TEIL2'].fillna('') + df_Lied['TEXT_TEIL3'].fillna('') + df_Lied['TEXT_TEIL4'].fillna('')


df_Chart_Position['DATUM_VON'] = pd.to_datetime(df_Chart_Position['DATUM_VON'])
df_Chart_Position['DATUM_BIS'] = pd.to_datetime(df_Chart_Position['DATUM_BIS'])
df_Chart_Position['DAUER'] = (df_Chart_Position['DATUM_BIS'] - df_Chart_Position['DATUM_VON']).dt.days.astype('int16')
df_Chart_Position['Jahr'] = df_Chart_Position['DATUM_BIS'].dt.year.astype('int16')
df_Chart_Position['Monat'] =  df_Chart_Position['DATUM_BIS'].dt.month.astype('int16')

Text Preprocessing

In [9]:
df_Lied['processed_Text'] = df_Lied['Text']

# ngram

#lemmatization
df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: lemmatize_string(text))

#Stemming
#df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: [snowStemmer.stem(token) for token in text])

# tokenize
df_Lied['processed_Text'] = df_Lied.processed_Text.apply(lambda text: nltk.word_tokenize(text))

#remove stopwords
df_Lied['processed_Text'] = df_Lied['processed_Text'].apply(lambda x: [item for item in x if item not in stopwords_list])

#remove numbers
df_Lied['processed_Text'] = df_Lied['processed_Text'].apply(lambda word_list : [re.sub('\w*\d\w*','', word) for word in word_list])

In [10]:
print(df_Lied['processed_Text'][0])

['LASS', 'STRAND', 'WEISSEN', 'BRAUCH', 'MAL', 'WIEDER', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIBEN', 'AI', 'UHR', 'FAELLT', 'BLAU', 'SEHEN', 'VERSINKEN', 'SETZEN', 'LANGSAM', 'WASSER', 'SPIEGELGLATT', 'BLINKEN', 'WOLLEN', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', '--', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', '--', 'STEHST', 'ZEIGST', 'ECHT', 'LICHTER', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'KOMMN', 'STEIGEN', 'LEUCHTEST', 'DIAMANT', 'YEHE', 'ALL', 'SALZ', 'HAUT', 'AI', 'SEH', 'GESTOCHEN', 'SCHARF', 'TUN', 'FAST', 'AUGE', 'WEH', 'ENDLICH', 'NOCH', 'NIE', 'SEHEN', 'YEHI', 'WOLLEN', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', '--', 'SEHNSUCHT', 'MORGEN', 'HEUTE', 'PERFEKT', '--', 'STEHST', 'ZEIGST', 'ECHT', 'LICHTER', 'STADT', 'VERBRANNT', 'LASS', 'STRAND', 'LASS', 'STRAND', 'WEISSEN', 'LASS', 'STRAND', 'YEAH', 'BRAUCHGRUND', 'BLEIBEN', 'AI', 'LASS', 'STRAND', 'WEISSEN', 'BRAUCH', 'MAL', 'WIEDER', 'MEER', 'SAND', 'YEAH', 'BRAUCH', 'MAL', 'WIEDERGRUND', 'BLEIBEN

In [11]:
print(df_Lied['Text'][0])

LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN ICH BRAUCH MAL WIEDER MEER UND SAND  YEAH BRAUCH MAL WIEDERGRUND ZU BLEIBEN  AI DEINE UHR FAELLT INS BLAU WIR SEHEN  WIE DIE ZEIT VERSINKT WIR SETZTEN LANGSAM AUF DAS WASSER SPIEGELGLATT UND BLINKT  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND    KOMM WIR STEIGEN AUS DU LEUCHTEST WIE AUS DIAMANT  YEHE MIT ALL DEM SALZ AUF DER HAUT  AI ICH SEH DICH GESTOCHEN SCHARF ES TUT FAST DEN AUGEN WEH UND ENDLICH BIST DU DA SO HABE ICH DICH NOCH NIE GESEHEN  YEHI  WIR WOLLEN DIE SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS SEHNSUCHT NACH MORGEN  WEIL ES HEUTE PERFEKT IS DU STEHST NEBEN MIR  ZEIGST MIR  DASS ES ECHT IST DIE LICHTER MEINER STADT SIND VERBRANNT LASS UNS ZUM STRAND   LASS UNS ZUM STRAND DU WEISST WELCHEN ICH MEIN LASS UNS ZUM STRAND  YEAH ICH BRAUCHGRUND ZU BLEIB

Data Selection

In [10]:
df_Lied.drop(['TEXT_TEIL1','TEXT_TEIL2', 'TEXT_TEIL3', 'TEXT_TEIL4', 'SPRACHE_DEUTSCH'], axis=1, inplace=True)
#df_Date = df_Chart_Position[['LIED_ID', 'DATUM_VON', 'DATUM_BIS', 'DAUER','Jahr', 'Monat']]
#df_Chart_Position.drop(['DATUM_VON', 'DATUM_BIS', 'Jahr', 'Monat'], axis=1, inplace=True)

#not used anymore:
#df_Chart_Position = df_Chart_Position.groupby('LIED_ID').agg({'POSITION':'mean','DAUER':'sum'}).reset_index()
#df_Chart_Position['POSITION'] = df_Chart_Position.POSITION.apply(lambda pos: round(pos))
#----

df_Lied.sort_values(by='ID', inplace=True)
df_Chart_Position.sort_values(by='LIED_ID', inplace=True)
df = pd.concat([df_Lied, df_Chart_Position], axis='columns')
df.drop('LIED_ID', axis=1, inplace=True)
print(df.head())

       ID       INTERPRET   TITEL  \
7219  408     Rumpelstilz  Kiosk    
7220  408     Rumpelstilz  Kiosk    
7763  425  Costa Cordalis  Anita    
7762  425  Costa Cordalis  Anita    
7761  425  Costa Cordalis  Anita    

                                                   Text  \
7219  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
7220  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
7763  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
7762  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
7761  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   

                                         processed_Text  POSITION  DATUM_VON  \
7219  [ALSO, SAMMLE, FUER, GUT, ZWECK, SAGEN, FRITZ,...        50 2009-03-06   
7220  [ALSO, SAMMLE, FUER, GUT, ZWECK, SAGEN, FRITZ,...         4 1984-10-22   
7763  [JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...        23 2003-08-25   
7762  [JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...        39 2003-07-28   
7761  [JVUIOUGIVTOH, F

In [11]:
df['ANZ_UNIQUE_WOERTER'] = list(len(set(word)) for word in df['processed_Text'])
df['LAENGE_LAENGSTES_WORT'] = list(len(max(set(word), key=len)) for word in df['processed_Text'])

MAX_RANK = 50
df['RANK_SCORE'] = MAX_RANK - df['POSITION'] + 1

profanity check

In [116]:
with open(RAW_DATA_PATH+'german-profanity-list.csv', newline='', encoding='UTF-8') as f:
    profanity_list = list(csv.reader(f))

profanity.load_words(flatten(profanity_list))

Percentage of Stopwords

In [12]:
df['Number_of_Stopwords'] = df.Text.str.split().apply(lambda x: len(set(x) & set(stopwords_list)))
df['Stopword_Percentage'] = df.Number_of_Stopwords.apply(lambda row: round(row/len(df['Text']), ndigits=5))
print(df['Stopword_Percentage'].head(20))

7219    0.00211
7220    0.00211
7763    0.00189
7762    0.00189
7761    0.00189
7760    0.00189
7743    0.00189
7744    0.00189
7745    0.00189
7746    0.00189
7747    0.00189
7748    0.00189
7749    0.00189
7751    0.00189
7752    0.00189
7753    0.00189
7754    0.00189
7755    0.00189
7756    0.00189
7757    0.00189
Name: Stopword_Percentage, dtype: float64


Sentiment Analysis

In [13]:
df['SUBJEKTIVITY'] = df.processed_Text.apply(lambda text: TextBlob(str(text)).sentiment.subjectivity)
df['POLARITY'] = df.processed_Text.apply(lambda text: TextBlob(str(text)).sentiment.polarity)

In [14]:
df.reset_index(inplace=True)
df.drop('index',axis=1, inplace=True)
print(df.head())

    ID       INTERPRET   TITEL  \
0  408     Rumpelstilz  Kiosk    
1  408     Rumpelstilz  Kiosk    
2  425  Costa Cordalis  Anita    
3  425  Costa Cordalis  Anita    
4  425  Costa Cordalis  Anita    

                                                Text  \
0  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
1  ALSO ER SAMMLE FUER EINEN GUTEN ZWECK  SAGT DE...   
2  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
3  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   
4  JVUIOUGIVTOH ICH FAND SIE IRGENDWO  ALLEIN IN ...   

                                      processed_Text  POSITION  DATUM_VON  \
0  [ALSO, SAMMLE, FUER, GUT, ZWECK, SAGEN, FRITZ,...        50 2009-03-06   
1  [ALSO, SAMMLE, FUER, GUT, ZWECK, SAGEN, FRITZ,...         4 1984-10-22   
2  [JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...        23 2003-08-25   
3  [JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...        39 2003-07-28   
4  [JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...        27 2003-07-28

LDA (BoW) - Title

Named Entity Recognition

In [32]:
test = [word for text in df_Lied['Text'] for word in nlp(text).ents]
print(test[0])

KeyboardInterrupt: 

Readability analysis

In [15]:
df.head()

,ID,INTERPRET,TITEL,Text,processed_Text,POSITION,DATUM_VON,DATUM_BIS,DAUER,Jahr,Monat,ANZ_UNIQUE_WOERTER,LAENGE_LAENGSTES_WORT,RANK_SCORE,Number_of_Stopwords,Stopword_Percentage,SUBJEKTIVITY,POLARITY
0,408,Rumpelstilz,Kiosk,ALSO ER SAMMLE FUER EINEN GUTEN ZWECK SAGT DE...,"[ALSO, SAMMLE, FUER, GUT, ZWECK, SAGEN, FRITZ,...",50,2009-03-06,2009-03-12,6,2009,3,82,15,1,48,0.00211,0.4,0.1
1,408,Rumpelstilz,Kiosk,ALSO ER SAMMLE FUER EINEN GUTEN ZWECK SAGT DE...,"[ALSO, SAMMLE, FUER, GUT, ZWECK, SAGEN, FRITZ,...",4,1984-10-22,1984-10-28,6,1984,10,82,15,47,48,0.00211,0.4,0.1
2,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",23,2003-08-25,2003-08-31,6,2003,8,74,12,28,43,0.00189,0.0,0.0
3,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",39,2003-07-28,2003-08-03,6,2003,8,74,12,12,43,0.00189,0.0,0.0
4,425,Costa Cordalis,Anita,JVUIOUGIVTOH ICH FAND SIE IRGENDWO ALLEIN IN ...,"[JVUIOUGIVTOH, FINDEN, IRGENDWO, ALLEIN, MEXIK...",27,2003-07-28,2003-08-03,6,2003,8,74,12,24,43,0.00189,0.0,0.0


Title Analysis

In [31]:
#titel_list = [word.upper() for word in list(set(df['TITEL']))]
#print(titel_list)
#lemmatize title
df['processed_Title'] = df.TITEL.apply(lambda titel: ' '.join([x.lemma_ for x in nlp(titel)]))
print(df['processed_Title'].head())

#tokenize title
tokenizer = RegexpTokenizer(r'\w+')
df['processed_Title'] = df.processed_Title.apply(lambda titel: [word for word in tokenizer.tokenize(titel)])

#remove stopwords
df['processed_Title'] = df.processed_Title.apply(lambda titel: [word for word in titel if word.upper() not in stopwords_list])

#remove numbers
df['processed_Title'] = df['processed_Title'].apply(lambda word_list : [re.sub('\w*\d\w*','', word) for word in word_list])

0    Kiosk
1    Kiosk
2    Anita
3    Anita
4    Anita
Name: processed_Title, dtype: object


Seasonal determination

In [32]:
print(df['processed_Title'].head(70))

0                                  [Kiosk]
1                                  [Kiosk]
2                                  [Anita]
3                                  [Anita]
4                                  [Anita]
                      ...                 
65                    [Mond, blühen, Rose]
66                    [Mond, blühen, Rose]
67    [Mußt, jetzt, grade, gehen, Lucille]
68    [Mußt, jetzt, grade, gehen, Lucille]
69    [Mußt, jetzt, grade, gehen, Lucille]
Name: processed_Title, Length: 70, dtype: object


Boosted Words -Analysis

Label Smoothing

Normalising

EDA

In [33]:
#create csv
#df_title_word_occurrence.to_csv('Data/processed/title_word_occurrence.csv')
#profanity Anteil

#sentiment Analyse
#Unique Wörter Anzahl Analyse

#Korreltaiton zwischen Interpret und Beliebtheit:
#df['INTERPRET'].value_counts(ascending=True, dropna=True, normalize=True)\
#.to_csv('Data/processed/Interpret_Occurrences_normalised.csv')

#df[['LAENGE_LAENGSTES_WORT', 'Jahr','ID','RANK_SCORE']].to_csv('Data/processed/Wort_LAENGE.csv')
#df[['ID','Number_of_Stopwords','Stopword_Percentage','Jahr', 'RANK_SCORE']].to_csv('Data/processed/Stoppwords.csv')

# Titel Analyse (länge des Titels, Wörter des Textes, ...)

df.to_csv('Data/processed/EDA3.csv')